In [13]:
import pandas as pd
df = pd.read_csv('Uitgebreide_VKM_dataset.csv')

In [14]:
df.isnull().sum(), df.duplicated().sum()

(id                         0
 name                       0
 shortdescription          20
 description                0
 content                    0
 studycredit                0
 location                   0
 contact_id                 0
 level                      0
 learningoutcomes           5
 Rood                     209
 Groen                    209
 Blauw                    209
 Geel                     209
 module_tags                0
 interests_match_score      0
 popularity_score           0
 estimated_difficulty       0
 available_spots            0
 start_date                 0
 dtype: int64,
 0)

Hier vervangen we alle waarden 'ntb', 'Nog niet bekend' en 'Nog te formuleren' naar 'Nog te bepalen' in ALLE kolommen.


Vervolgens verwijderen we de onnodige kolommen die geen impact hebben op de keuze van de top 3 (bijv. 'Rood', 'Groen', 'Blauw', 'Geel').


Tot slot tonen we nog eens het aantal null-waarden en duplicaten om te bevestigen dat alles goed is gegaan.

In [15]:
for col in df.columns:
    if df[col].dtype == 'object':
        for i in df.index:
            cell_value = str(df.loc[i, col]).strip()
            # Als de waarde leeg is (NaN of lege string) of een van de te vervangen waarden
            if pd.isna(df.loc[i, col]) or cell_value == '' or cell_value.lower() in ['ntb', 'nog niet bekend', 'nog te formuleren']:
                df.loc[i, col] = 'Nog te bepalen'

df.drop(['Rood', 'Groen', 'Blauw', 'Geel'], axis=1, inplace=True)
df.isnull().sum(), df.duplicated().sum()

(id                       0
 name                     0
 shortdescription         0
 description              0
 content                  0
 studycredit              0
 location                 0
 contact_id               0
 level                    0
 learningoutcomes         0
 module_tags              0
 interests_match_score    0
 popularity_score         0
 estimated_difficulty     0
 available_spots          0
 start_date               0
 dtype: int64,
 0)

Controleer hoeveel 'Nog te bepalen' waarden er aanwezig zijn in de kolom `shortdescription`.


Voor rijen waarbij `shortdescription` nog 'Nog te bepalen' is, vullen we deze op met de samengestelde tekst van `description` + `content`.


Na het invullen tonen we opnieuw het aantal resterende 'Nog te bepalen' waarden om te verifiëren dat de vervanging is uitgevoerd.

In [16]:
print("Aantal 'Nog te bepalen' in shortdescription:", (df['shortdescription'] == 'Nog te bepalen').sum())

for i in df.index:
    if df.loc[i, 'shortdescription'] == 'Nog te bepalen':
        description = str(df.loc[i, 'description']) if pd.notna(df.loc[i, 'description']) else ''
        content = str(df.loc[i, 'content']) if pd.notna(df.loc[i, 'content']) else ''
        df.loc[i, 'shortdescription'] = description + ' ' + content

print("\nAantal 'Nog te bepalen' na aanpassing:", (df['shortdescription'] == 'Nog te bepalen').sum())
print("Klaar!")

Aantal 'Nog te bepalen' in shortdescription: 30

Aantal 'Nog te bepalen' na aanpassing: 0
Klaar!
